# DeepFM Ranking (RecBole)


- Loads configuration from `deepfm_config.yaml` and optional env-var overrides
- Builds RecBole dataset/dataloaders, trains DeepFM, evaluates on the test split
- Saves the best checkpoint + JSON metrics under a configurable directory




In [1]:
import json
import os
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.context_aware_recommender import DeepFM
from recbole.trainer import Trainer
from recbole.utils import init_seed
from recbole.utils.case_study import full_sort_topk

try:
    import faiss
    FAISS_AVAILABLE = True
except ImportError:
    faiss = None
    FAISS_AVAILABLE = False
    print("FAISS not found. Falling back to numpy search for candidates.")

print("RecBole imported successfully")



RecBole imported successfully


In [2]:
DATASET = os.environ.get("RECO_DATASET", "amazon-beauty")
CHECKPOINT_DIR = os.environ.get("CHECKPOINT_DIR", "saved_models/deepfm")
DATA_PATH = os.environ.get("DATA_PATH") or os.environ.get("DATA_DIR")
GPU_ID = int(os.environ.get("GPU_ID", 0))
SEED = int(os.environ.get("SEED", 42))
USE_GPU = os.environ.get("USE_GPU", "true").lower() != "false"

config_overrides = {
    "checkpoint_dir": CHECKPOINT_DIR,
    "gpu_id": GPU_ID,
    "use_gpu": USE_GPU,
    "seed": SEED,
}
if DATA_PATH:
    config_overrides["data_path"] = DATA_PATH

print("Dataset:", DATASET)
print("Checkpoint dir:", CHECKPOINT_DIR)
print("GPU ID:", GPU_ID)
print("Use GPU:", USE_GPU)
print("Seed:", SEED)
if DATA_PATH:
    print("Data path override:", DATA_PATH)



Dataset: amazon-beauty
Checkpoint dir: saved_models/deepfm
GPU ID: 0
Use GPU: True
Seed: 42


In [3]:
def build_config():
    config = Config(
        model="DeepFM",
        dataset=DATASET,
        config_file_list=["deepfm_config.yaml"],
        config_dict=config_overrides,
    )
    print("Using device:", config["device"])
    return config


def run_deepfm_training():
    config = build_config()
    init_seed(config["seed"], config["reproducibility"])

    print("Creating dataset")
    dataset = create_dataset(config)
    print(f"      Dataset created: {dataset}")
    
    print("Preparing train/valid/test data")
    train_data, valid_data, test_data = data_preparation(config, dataset)
    print(f"      Train batches: {len(train_data)}, Valid batches: {len(valid_data)}, Test batches: {len(test_data)}")

    print("Creating DeepFM model")
    model = DeepFM(config, dataset).to(config["device"])
    print(f"      Model created: {model.__class__.__name__}")
    
    print("Initializing trainer")
    trainer = Trainer(config, model)
    
    print("Starting training")
    import sys; sys.stdout.flush()
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    
    # Load best checkpoint so we can reuse the model for re-ranking
    best_ckpt = trainer.saved_model_file
    if os.path.exists(best_ckpt):
        state = torch.load(best_ckpt, map_location=config["device"])
        model.load_state_dict(state["state_dict"])
        if "other_parameter" in state:
            model.load_other_parameter(state["other_parameter"])
        model.to(config["device"])
        model.eval()
    else:
        print("Warning: best checkpoint not found. Using current model state.")

    print("Evaluating on test set")
    try:
        test_result = trainer.evaluate(test_data, load_best_model=True)
    except FileNotFoundError:
        print("Warning: No best model checkpoint found. Evaluating current model state.")
        test_result = trainer.evaluate(test_data, load_best_model=False)

    results = {
        "best_valid_score": best_valid_score,
        "best_valid_result": best_valid_result,
        "test_result": test_result,
    }

    os.makedirs(config["checkpoint_dir"], exist_ok=True)
    output_path = os.path.join(config["checkpoint_dir"], "deepfm_results.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    print("Best validation score:", best_valid_score)
    print("Best validation result:", best_valid_result)
    print("Test result:", test_result)
    print("Saved results to:", output_path)
    return {
        "results": results,
        "model": model,
        "dataset": dataset,
        "train_data": train_data,
        "valid_data": valid_data,
        "test_data": test_data,
        "config": config,
    }



In [4]:
%%time
artifacts = run_deepfm_training()
deepfm_results = artifacts["results"]
deepfm_model = artifacts["model"]
deepfm_dataset = artifacts["dataset"]
deepfm_test_data = artifacts["test_data"]
deepfm_config = artifacts["config"]



Using device: cuda
Creating dataset


/home/guo.yunyu/.conda/envs/recsys_gpu/lib/python3.10/site-packages/recbole/data/dataset/dataset.py:501: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[field].fillna(value="", inplace=True)
/home/guo.yunyu/.conda/envs/recsys_gpu/lib/python3.10/site-packages/recbole/data/dataset/dataset.py:1217: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  split_point = np.cumsum(feat[field].agg(len))[:-1]
/home/guo.yunyu/.conda/envs/rec

      Dataset created: amazon-beauty
The number of users: 1210272
Average actions of users: 1.6715842980621696
The number of items: 259205
Average actions of items: 8.115848423822781
The number of inters: 2023070
The sparsity of the dataset: 99.99935511162327%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'title', 'sales_type', 'sales_rank', 'categories', 'price', 'brand', 'label']
Preparing train/valid/test data
      Train batches: 753, Valid batches: 4084, Test batches: 8969
Creating DeepFM model
      Model created: DeepFM
Initializing trainer
Starting training


/home/guo.yunyu/.conda/envs/recsys_gpu/lib/python3.10/site-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)
/tmp/ipykernel_3332452/1420480766.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend 

Evaluating on test set


/home/guo.yunyu/.conda/envs/recsys_gpu/lib/python3.10/site-packages/recbole/trainer/trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(che

Best validation score: 0.9984
Best validation result: OrderedDict([('recall@10', 0.9984), ('recall@20', 0.9996), ('ndcg@10', 0.8154), ('ndcg@20', 0.8158), ('hit@10', 1.0), ('hit@20', 1.0)])
Test result: OrderedDict([('recall@10', 0.9991), ('recall@20', 0.9998), ('ndcg@10', 0.7915), ('ndcg@20', 0.7917), ('hit@10', 1.0), ('hit@20', 1.0)])
Saved results to: saved_models/deepfm/deepfm_results.json
CPU times: user 13min 32s, sys: 7.19 s, total: 13min 39s
Wall time: 13min 4s


# recall@10	0.9984(train)	0.9991(test)	99.8% of the time the true item appears in the top 10 candidates
# ndcg@10	0.8154(train)	0.7915(test)	Measures ranking quality (weights top positions). Values around 0.8 are very high
# hit@10	1.0(train)	1.0(test)	The true item is in the top 10 for every user (100%)
# The DeepFM model is performing extremely well under the current evaluation (more than 99% recall with random negative sampling).
# Metrics indicate the model consistently ranks the true item in the top 10.

## Candidate Generation via Two-Tower + FAISS

reuse the RecBole `.inter` splits to train a lightweight two-tower model, build a FAISS index over item embeddings, and stage top candidates per user. These candidates feed into DeepFM for re-ranking.


In [5]:
INTER_DIR = Path("dataset/amazon-beauty")
TRAIN_INTER = INTER_DIR / "amazon-beauty-train.inter"
VALID_INTER = INTER_DIR / "amazon-beauty-valid.inter"
TEST_INTER = INTER_DIR / "amazon-beauty-test.inter"

assert TRAIN_INTER.exists(), "Missing RecBole split files. Run the dataset download first."

use_cols = ["user_id", "item_id", "label", "timestamp"]
train_df = pd.read_csv(TRAIN_INTER, sep="\t", usecols=use_cols)
valid_df = pd.read_csv(VALID_INTER, sep="\t", usecols=use_cols)
test_df = pd.read_csv(TEST_INTER, sep="\t", usecols=use_cols)

# consistent string tokens (matches RecBole dataset tokens)
for df in (train_df, valid_df, test_df):
    df["user_id"] = df["user_id"].astype(str)
    df["item_id"] = df["item_id"].astype(str)

def build_id_mappings(df_list):
    unique_users = pd.concat([df["user_id"] for df in df_list]).unique()
    unique_items = pd.concat([df["item_id"] for df in df_list]).unique()
    user2idx = {u: idx for idx, u in enumerate(unique_users)}
    item2idx = {i: idx for idx, i in enumerate(unique_items)}
    idx2item = {idx: item for item, idx in item2idx.items()}
    return user2idx, item2idx, idx2item

user2idx, item2idx, idx2item = build_id_mappings([train_df, valid_df, test_df])
num_users, num_items = len(user2idx), len(item2idx)
print(f"Two-tower universe -> users: {num_users:,}, items: {num_items:,}")



Two-tower universe -> users: 1,210,271, items: 249,274


In [6]:
class TwoTowerDataset(Dataset):
    def __init__(self, df, user2idx, item2idx, num_items, neg_ratio=1, seed=42):
        self.user = df["user_id"].map(user2idx).values
        self.item = df["item_id"].map(item2idx).values
        self.label = df["label"].values
        self.num_items = num_items
        self.neg_ratio = neg_ratio
        self.rng = np.random.default_rng(seed)

    def __len__(self):
        return len(self.user) * (1 + self.neg_ratio)

    def __getitem__(self, idx):
        base_idx = idx // (1 + self.neg_ratio)
        is_pos = (idx % (1 + self.neg_ratio)) == 0
        u = self.user[base_idx]
        if is_pos:
            i = self.item[base_idx]
            y = 1.0
        else:
            i = self.rng.integers(0, self.num_items)
            y = 0.0
        return (
            torch.tensor(u, dtype=torch.long),
            torch.tensor(i, dtype=torch.long),
            torch.tensor(y, dtype=torch.float32),
        )


class TwoTowerModel(nn.Module):
    def __init__(self, num_users, num_items, emb_dim=128):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        # Initialize embeddings
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)

    def forward(self, users, items):
        u = self.user_emb(users)
        i = self.item_emb(items)
        return (u * i).sum(dim=-1)

"""
    Train a two-tower retrieval model.
    
    """
def train_two_tower(train_df, user2idx, item2idx, num_items, epochs=10, batch_size=4096, lr=5e-4, neg_ratio=4, emb_dim=128):
    
    dataset = TwoTowerDataset(train_df, user2idx, item2idx, num_items, neg_ratio=neg_ratio)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training two-tower on {device}, {len(dataset):,} samples, emb_dim={emb_dim}, neg_ratio={neg_ratio}")
    
    model = TwoTowerModel(len(user2idx), num_items, emb_dim=emb_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for users, items, labels in loader:
            users = users.to(device)
            items = items.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            logits = model(users, items)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * len(users)
        avg_loss = running_loss / len(dataset)
        print(f"Two-tower epoch {epoch+1}/{epochs} - loss {avg_loss:.4f}")
    return model



In [7]:
def build_faiss_index(item_embeddings):
    dim = item_embeddings.shape[1]
    if FAISS_AVAILABLE:
        faiss.normalize_L2(item_embeddings)
        cpu_index = faiss.IndexFlatIP(dim)
        gpu_enabled = False
        if hasattr(faiss, "get_num_gpus") and faiss.get_num_gpus() > 0:
            try:
                res = faiss.StandardGpuResources()
                gpu_id = int(os.environ.get("GPU_ID", 0))
                index = faiss.index_cpu_to_gpu(res, gpu_id, cpu_index)
                gpu_enabled = True
            except Exception as err:  # pragma: no cover
                print(f"Warning: falling back to CPU FAISS ({err})")
                index = cpu_index
        else:
            index = cpu_index
        index.add(item_embeddings.astype(np.float32))
        return index, gpu_enabled
    else:
        norms = np.linalg.norm(item_embeddings, axis=1, keepdims=True) + 1e-9
        normalized = item_embeddings / norms
        class NumpyIndex:
            def __init__(self, emb):
                self.emb = emb
            def search(self, queries, k):
                sims = queries @ self.emb.T
                idx = np.argpartition(sims, -k, axis=1)[:, -k:]
                part = np.take_along_axis(sims, idx, axis=1)
                order = np.argsort(part, axis=1)[:, ::-1]
                top_idx = np.take_along_axis(idx, order, axis=1)
                top_scores = np.take_along_axis(sims, top_idx, axis=1)
                return top_scores, top_idx
        return NumpyIndex(normalized), False


def generate_candidates(model, history_df, test_df, user2idx, item2idx, idx2item, candidate_topk=100, min_session_len=5, max_users=5000):
    device = next(model.parameters()).device
    model.eval()
    user_emb = model.user_emb.weight.detach().cpu().numpy()
    item_emb = model.item_emb.weight.detach().cpu().numpy()

    # string tokens so stay consistent with RecBole and the .inter files
    history_df = history_df.copy()
    test_df = test_df.copy()
    history_df["user_id"] = history_df["user_id"].astype(str)
    test_df["user_id"] = test_df["user_id"].astype(str)
    test_df["item_id"] = test_df["item_id"].astype(str)

    index, used_faiss = build_faiss_index(item_emb.copy())
    print("FAISS index" if used_faiss else "Numpy index", "will serve retrieval")

    hist_counts = history_df.groupby("user_id").size()
    eligible_users = hist_counts[hist_counts >= min_session_len].index

    test_pos = test_df[test_df["label"] == 1]
    test_pos = test_pos[test_pos["user_id"].isin(eligible_users)]
    if max_users:
        test_pos = test_pos.groupby("user_id").head(1)
        unique_users = test_pos["user_id"].unique()
        if len(unique_users) > max_users:
            sampled = np.random.choice(unique_users, size=max_users, replace=False)
            test_pos = test_pos[test_pos["user_id"].isin(sampled)]

    candidate_dict = {}
    metrics_hits = {k: 0 for k in (5, 10, 20)}
    total = 0

    for row in test_pos.itertuples(index=False):
        user_token = str(row.user_id)
        item_token = str(row.item_id)
        u_idx = user2idx.get(user_token)
        i_idx = item2idx.get(item_token)
        if u_idx is None or i_idx is None:
            continue
        user_vec = user_emb[u_idx]
        user_vec = user_vec / (np.linalg.norm(user_vec) + 1e-9)
        scores, idxs = index.search(user_vec.reshape(1, -1), candidate_topk)
        top_items = [str(idx2item[idx]) for idx in idxs[0]]
        candidate_dict[user_token] = top_items

        total += 1
        for k in metrics_hits.keys():
            if item_token in top_items[:k]:
                metrics_hits[k] += 1

    if total == 0:
        raise RuntimeError("No eligible users found for two-tower retrieval")

    retrieval_metrics = {f"recall@{k}": metrics_hits[k] / total for k in metrics_hits}
    retrieval_metrics["num_users"] = total
    return candidate_dict, retrieval_metrics



In [8]:
%%time
history_df = pd.concat([train_df, valid_df], ignore_index=True)


two_tower_model = train_two_tower(
    train_df, user2idx, item2idx, num_items,
    epochs=10,       
    batch_size=4096,
    lr=5e-4,
    neg_ratio=4,     
    emb_dim=128      
)

candidate_dict, retrieval_metrics = generate_candidates(
    two_tower_model,
    history_df,
    test_df,
    user2idx,
    item2idx,
    idx2item,
    candidate_topk=200,
    min_session_len=5,
    max_users=5000,
)
print("Retrieval metrics:", retrieval_metrics)



Training two-tower on cuda, 7,706,605 samples, emb_dim=128, neg_ratio=4
Two-tower epoch 1/10 - loss 0.6927
Two-tower epoch 2/10 - loss 0.6585
Two-tower epoch 3/10 - loss 0.5182
Two-tower epoch 4/10 - loss 0.3772
Two-tower epoch 5/10 - loss 0.2989
Two-tower epoch 6/10 - loss 0.2577
Two-tower epoch 7/10 - loss 0.2287
Two-tower epoch 8/10 - loss 0.2029
Two-tower epoch 9/10 - loss 0.1782
Two-tower epoch 10/10 - loss 0.1544
FAISS index will serve retrieval
Retrieval metrics: {'recall@5': 0.0026, 'recall@10': 0.0034, 'recall@20': 0.0058, 'num_users': 5000}
CPU times: user 17min 58s, sys: 11.2 s, total: 18min 9s
Wall time: 25min 47s


Out of 5000 test users, only 29 users (0.58%) had their ground-truth item in the top-200 candidates

Reason: 
	                
249K items to search:	Finding 1 needle in 249K haystack

99.999% sparsity:	Each user has only ~1.7 interactions

Simple model:	Just dot-product of embeddings

Cold users:	Many users have minimal history

In [9]:
import sys

"""
    Re-rank two-tower candidates using DeepFM.
    candidate_dict keys/values are RecBole internal indices 
    """
def rerank_candidates_with_deepfm(model, dataset, config, candidate_dict):
    
    from recbole.data.interaction import Interaction

    device = config["device"]
    model.eval()

    uid_field = dataset.uid_field
    iid_field = dataset.iid_field
    num_users = dataset.num(uid_field)
    num_items = dataset.num(iid_field)

    n_cands = len(next(iter(candidate_dict.values())))
    print(f"Re-ranking {len(candidate_dict)} users x ~{n_cands} candidates")
    print(f"RecBole vocab: {num_users} users, {num_items} items")
    sys.stdout.flush()

    reranked = {}
    skipped = 0
    skip_reasons = {"user_out_of_range": 0, "no_valid_items": 0}

    with torch.no_grad():
        for i, (user_key, item_keys) in enumerate(candidate_dict.items()):
            if i % 1000 == 0:
                print(f"  User {i}/{len(candidate_dict)}...")
                sys.stdout.flush()

            # candidate_dict keys are RecBole internal indices (ints or str of ints)
            try:
                uid_internal = int(user_key)
            except (ValueError, TypeError):
                skipped += 1
                skip_reasons["user_out_of_range"] += 1
                continue
            if not (0 <= uid_internal < num_users):
                skipped += 1
                skip_reasons["user_out_of_range"] += 1
                continue

            valid_pairs = []
            for item_key in item_keys:
                try:
                    iid_internal = int(item_key)
                except (ValueError, TypeError):
                    continue
                if 0 <= iid_internal < num_items:
                    valid_pairs.append((iid_internal, item_key))
            if not valid_pairs:
                skipped += 1
                skip_reasons["no_valid_items"] += 1
                continue

            internal_iids, original_keys = zip(*valid_pairs)
            n = len(internal_iids)
            user_tensor = torch.full((n,), uid_internal, dtype=torch.long, device=device)
            item_tensor = torch.tensor(internal_iids, dtype=torch.long, device=device)

            interaction = Interaction({uid_field: user_tensor, iid_field: item_tensor})
            interaction = dataset.join(interaction)
            interaction = interaction.to(device)

            scores = model.predict(interaction).cpu().numpy()
            ranked_idx = np.argsort(-scores)
            reranked[user_key] = [original_keys[j] for j in ranked_idx]

    print(f"Done. Re-ranked {len(reranked)} users. Skipped {skipped}.")
    if skipped:
        print("Skip reasons:", skip_reasons)
    return reranked


# Re-rank only two-tower candidates, keeps IDs consistent with RecBole tokens
deepfm_reranked = rerank_candidates_with_deepfm(
    deepfm_model, deepfm_dataset, deepfm_config, candidate_dict
)
print(f"DeepFM re-ranked {len(deepfm_reranked)} users")



Re-ranking 5000 users x ~200 candidates
RecBole vocab: 1210272 users, 259205 items
  User 0/5000...
  User 1000/5000...
  User 2000/5000...
  User 3000/5000...
  User 4000/5000...
Done. Re-ranked 5000 users. Skipped 0.
DeepFM re-ranked 5000 users


In [10]:
# === Evaluation ===

def evaluate_ranked_lists(ranked_dict, test_df, topks=(5, 10, 20)):
    """Evaluate ranked recommendations against test set ground truth."""
    pos_df = test_df[test_df["label"] == 1]
    pos_df = pos_df[pos_df["user_id"].isin(ranked_dict.keys())]
    metrics = {f"hit@{k}": 0 for k in topks}
    metrics.update({f"recall@{k}": 0 for k in topks})
    metrics.update({f"ndcg@{k}": 0.0 for k in topks})
    total = 0
    for row in pos_df.itertuples(index=False):
        ranked = ranked_dict.get(row.user_id)
        if not ranked:
            continue
        total += 1
        for k in topks:
            topk = ranked[:k]
            if row.item_id in topk:
                metrics[f"hit@{k}"] += 1
                metrics[f"recall@{k}"] += 1
                rank = topk.index(row.item_id)
                metrics[f"ndcg@{k}"] += 1 / np.log2(rank + 2)
    if total == 0:
        raise RuntimeError("No overlapping users for ranked evaluation.")
    for k in topks:
        metrics[f"hit@{k}"] /= total
        metrics[f"recall@{k}"] /= total
        metrics[f"ndcg@{k}"] /= total
    metrics["num_users"] = total
    return metrics

# top-20 for final evaluation (stored as string tokens)
reranked_candidates = {str(u): [str(i) for i in items[:20]] for u, items in deepfm_reranked.items()}

print(f"Evaluating {len(reranked_candidates)} users")
print(f"Sample user keys: {list(reranked_candidates.keys())[:3]}")

# test_df types
test_eval_df = test_df.copy()
test_eval_df["user_id"] = test_eval_df["user_id"].astype(str)
test_eval_df["item_id"] = test_eval_df["item_id"].astype(str)
print(f"Test df user_id sample: {test_eval_df['user_id'].head(3).tolist()}")

pipeline_metrics = evaluate_ranked_lists(reranked_candidates, test_eval_df, topks=(5, 10, 20))
print("\n=== Pipeline Results (TwoTower → DeepFM) ===")
for k, v in pipeline_metrics.items():
    print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")



Evaluating 5000 users
Sample user keys: ['40', '62', '141']
Test df user_id sample: ['1', '2', '5']

=== Pipeline Results (TwoTower → DeepFM) ===
  hit@5: 0.0025
  hit@10: 0.0039
  hit@20: 0.0071
  recall@5: 0.0025
  recall@10: 0.0039
  recall@20: 0.0071
  ndcg@5: 0.0016
  ndcg@10: 0.0020
  ndcg@20: 0.0028
  num_users: 5883


TwoTower: recall@20 = 0.0058

TwoTower+DeepFM: recall@20 = 0.0071 (+22% improvement)

In [11]:
def extract_metric(result_dict, metric_name):
    for key, value in result_dict.items():
        if key.lower() == metric_name.lower():
            return float(value)
    return np.nan

comparison_rows = []
for k in (5, 10, 20):
    comparison_rows.append({
        "metric": f"recall@{k}",
        "DeepFM_full": extract_metric(deepfm_results["test_result"], f"Recall@{k}"),
        "TwoTower": retrieval_metrics.get(f"recall@{k}", np.nan),
        "TwoTower+DeepFM": pipeline_metrics.get(f"recall@{k}", np.nan),
    })
    comparison_rows.append({
        "metric": f"ndcg@{k}",
        "DeepFM_full": extract_metric(deepfm_results["test_result"], f"NDCG@{k}"),
        "TwoTower": np.nan,  # retrieval computed only recall@K
        "TwoTower+DeepFM": pipeline_metrics.get(f"ndcg@{k}", np.nan),
    })
comparison_rows.append({
    "metric": "num_users_eval",
    "DeepFM_full": len(test_df[test_df["label"] == 1]["user_id"].unique()),
    "TwoTower": retrieval_metrics["num_users"],
    "TwoTower+DeepFM": pipeline_metrics["num_users"],
})
comparison_df = pd.DataFrame(comparison_rows)
display(comparison_df)



,metric,DeepFM_full,TwoTower,TwoTower+DeepFM
0,recall@5,NaN,0.0026,0.002550
1,ndcg@5,NaN,NaN,0.001574
2,recall@10,0.9991,0.0034,0.003910
3,ndcg@10,0.7915,NaN,0.002002
4,recall@20,0.9998,0.0058,0.007139
5,ndcg@20,0.7917,NaN,0.002822
6,num_users_eval,322870.0000,5000.0000,5883.000000


Althought DeepFM has significantly hight recall@10, but this is not a fair comparison

DeepFM_full: Evaluated against 5 random negatives (easy, due to time constraint)

TwoTower + DeepFM: Evaluated against 249K items (hard)

The pipeline is technically correct, but the two-tower retrieval is too weak for this extremely sparse dataset. 